In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [ ]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [ ]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

In [ ]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis = 1)
    
    return df

X = add_interactions(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify = y)

In [ ]:
test = X_test
test['cst_id_di'] = y_test.index
test = test.set_index('cst_id_di')
test['MRC_ID_DI'] = y_test

X_test = test.drop(columns = ['MRC_ID_DI'], axis=1)
y_test = test['MRC_ID_DI']



train = X_train
train['cst_id_di'] = y_train.index
train = train.set_index('cst_id_di')
train['MRC_ID_DI'] = y_train

train_0 = train[train['MRC_ID_DI'] == 0].sample(frac=1)
train_1 = train[train['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(train_0) if len(train_0) < len(train_1) else len(train_1)

train_f = pd.concat([train_0.head(sample_size), train_1.head(sample_size)]).sample(frac=1)

X_train = train_f.drop(columns = ['MRC_ID_DI'], axis=1)
y_train = train_f['MRC_ID_DI']

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X_train.columns[i] for i in indices_selected]

X_train = X_train[colnames_selected]
X_test = X_test[colnames_selected]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify = y_train)

In [ ]:
X_train[X_train.columns] = (X_train[X_train.columns]+1)/2*255
X_test[X_test.columns] = (X_test[X_test.columns]+1)/2*255
X_val[X_val.columns] = (X_val[X_val.columns]+1)/2*255

In [ ]:
repetitions = 331

from PIL import Image
import numpy as np
for ind, k in enumerate(y_train.index):
    print(ind)
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    
    an_array = X_train.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((331,331, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    if y_train.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image6', 'train', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image6', 'train', '1', str(k) + '.png')
    img.save(img_name)

In [ ]:
for ind, k in enumerate(y_test.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    
    an_array = X_test.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((331,331, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    if y_test.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image6', 'test', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image6', 'test', '1', str(k) + '.png')
    img.save(img_name)

In [ ]:
for ind, k in enumerate(y_val.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    
    an_array = X_val.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((331,331, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    if y_val.iloc[ind] == 0:
        img_name = os.path.join(os.getcwd(), 'image6', 'val', '0', str(k) + '.png')
    else:
        img_name = os.path.join(os.getcwd(), 'image6', 'val', '1', str(k) + '.png')
    img.save(img_name)